In [1]:
from model.wavenet import WaveNetPedal

In [2]:
configs = {'dilation_rates':[2,4,8], 'num_filters': 16, 'kernel_size': 4, 'stride': 2}
x = WaveNetPedal(configs)

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 1)]    0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, None, 16)     32          input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, None, 16)     1040        conv1d[0][0]                     
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, None, 16)     1040        conv1d[0][0]                     
_______________________________________________________________________________________

In [ ]:
x(tf.random.normal())